In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


In [2]:
import sys
sys.path.insert(0,'../..')
import omama as O
import pickle
from hyperopt import hp, rand, fmin, Trials
from pyod.models.cblof import CBLOF

In [3]:
# load data and labels

with open('/raid/mpsych/datasetBSTAR.pkl','rb') as f:
    datasetA = pickle.load(f)
    
with open('/raid/mpsych/datasetBSTAR_labels.pkl', 'rb') as f1:
    labels = np.asarray(pickle.load(f1))

In [4]:
# determine the hyperparameter space

param_grid = {
    'contamination': hp.uniform('contamination', 0.07, .1),
    'n_clusters': hp.uniformint('n_clusters', 1, 200),
    'alpha': hp.uniform('alpha', 0.0, 1.0),
    'beta': hp.uniform('beta', 0.0, 10.0),
    'use_weights': hp.choice('use_weights', [True, False]),
    'check_estimator': hp.choice('check_estimator', [True, False]),
    'n_jobs': hp.uniformint('n_jobs', 1, 4),
    'clustering_estimator': hp.choice('clustering_estimator', [None, 'clustering_estimator']),
}

In [9]:
def opt(params):
    param_dict = {
        'contamination': params['contamination'],
        'n_clusters': int(params['n_clusters']),
        'alpha': params['alpha'],
        'beta': params['beta'],
        'use_weights': params['use_weights'],
        'check_estimator': params['check_estimator'],
        'n_jobs': int(params['n_jobs']),
        'clustering_estimator': params['clustering_estimator'],
    }
    clf = self.algorithm(**params)
    clf.fit(self.X_train, self.y_train)
    
    score = -cross_val_score(clf,
                             self.X_train,
                             self.y_train,
                             cv=self.cv,
                             n_jobs=self.n_jobs,
                             scoring=self.scoring).mean()

In [5]:
hyperOpt = O.HyperTuner(
    algorithm=CBLOF,
    param_space=param_grid,
    feature_type='hist',
    norm_type='gaussian',
    data=datasetA,
    gt_labels=labels,
    max_evals=60,
    opt
)

In [6]:
hyperOpt.optimize()

  0%|                                                                                                                                                                  | 0/60 [00:00<?, ?trial/s, best loss=?]


AttributeError: 'int' object has no attribute 'integers'

In [8]:
hyperOpt.display_best_params()

Best parameters:
contamination: 0.07236974600054445
leaf_size: 83.0
method: 2.0
metric: 3.0
n_jobs: 2.0
n_neighbors: 44.0
p: 6.0
radius: 2.0


In [18]:
hyperOpt.print_param_space()

Parameter space:
contamination: ['0.07', '0.1']
n_neighbors: ['1', '50', '1.0']
method: ['largest', 'mean', 'median']
radius: ['0.0', '5.0', '0.1']
leaf_size: ['1', '100', '1.0']
metric: ['minkowski', 'cityblock', 'euclidean', 'l1', 'l2', 'manhattan']
p: ['1', '10', '1.0']
n_jobs: ['1', '4', '1.0']


In [35]:
print(type(hyperOpt.param_space))

<class 'dict'>
